In [25]:
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time

In [26]:
pd.set_option('display.max_columns', None)

- - - 

# Trapist

In [27]:
todos_produtos=pd.DataFrame()
todos_faturamentos=pd.DataFrame()
todas_recargas=pd.DataFrame()

In [28]:
#Tardezinha 15 de Julho de 2023
#Trapist 9 de março de 2024
# SOweto 27 de Julho de 2024



start_date = '2024-03-08'
end_date = '2024-03-10'
id_loja='3123a77b-2730-4b44-ab69-24bba879c56b'


token = "hlqFYMUrmLX9NGTd07jC5ZjxLRKsOfqh"







#PRODUTOS


api_url = "https://api.zigcore.com.br/integration/erp/saida-produtos"


params = {
    'dtinicio': start_date,
    'dtfim': end_date,
    'loja': id_loja, #ID que já achamos
    'refunded': 'false'
}


headers = {
    "Authorization": f"{token}",
    "Content-Type": "application/json"
}


response_produtos = requests.get(api_url, headers=headers, params=params)

produtos=pd.DataFrame(response_produtos.json())
produtos= produtos[["transactionDate","unitValue","count","productName","productCategory","eventName","employeeName"]]
produtos=produtos.rename(columns={"transactionDate":"Data e Hora","unitValue":"Soma de price","count":"Quantidade","productName":"product_name","eventName":"Evento","employeeName":"PDV"})

#Criando coluna para avaliar as vendas durante o evento
produtos["Data e Hora"]=pd.to_datetime(produtos["Data e Hora"])
produtos["Hora"]=produtos["Data e Hora"].dt.strftime('%Y-%m-%d %H')

#Formatando a coluna de Soma de price
produtos["Soma de price"]=produtos["Soma de price"]/100

todos_produtos=pd.concat([todos_produtos,produtos], ignore_index=True)





#FATURAMENTO

api_url = "https://api.zigcore.com.br/integration/erp/faturamento"

params = {
    'dtinicio': start_date,
    'dtfim': end_date,
    'loja': id_loja
}


headers = {
    "Authorization": f"{token}",
    "Content-Type": "application/json"
}


response_faturamento = requests.get(api_url, headers=headers, params=params)

faturamento=pd.DataFrame(response_faturamento.json())
faturamento=faturamento[["paymentName","value"]]


#Corrigindo os valores
faturamento["value"]=faturamento["value"]/100


todos_faturamentos=pd.concat([todos_faturamentos,faturamento], ignore_index=True)




#RECARGAS

api_url = "https://api.zigcore.com.br/integration/erp/recharges"
token = "hlqFYMUrmLX9NGTd07jC5ZjxLRKsOfqh"

params = {
    'dtinicio': start_date,
    'dtfim': end_date,
    'loja': id_loja
}


headers = {
    "Authorization": f"{token}",
    "Content-Type": "application/json"
}


response_recarga = requests.get(api_url, headers=headers, params=params)

recarga=pd.DataFrame(response_recarga.json())

recarga=recarga[["date","author","totalValue","userDocument","userDocumentType","userName"]]

#Corrigindo os valores
recarga["totalValue"]= recarga["totalValue"]/100

#Valores das datas
recarga["date"]=pd.to_datetime(recarga["date"])
recarga["Dia + Hora"]=recarga["date"].dt.strftime('%Y-%m-%d %H')

todas_recargas=pd.concat([todas_recargas,recarga], ignore_index=True)






In [29]:
todos_produtos["valor_total"]=todos_produtos["Soma de price"]*todos_produtos["Quantidade"]


In [30]:
todos_produtos["Dia Hora"]=todos_produtos["Data e Hora"].dt.strftime('%Y-%m-%d %H:00:00')
todos_produtos.rename(columns={
    "product_name":"produto",
    "productCategory":"categoria_produto",
    "Quantidade":"quantidade",
    "Soma de price":"valor_unitario",
    "Evento":"nome_evento"



}, inplace=True)

In [31]:
todos_produtos

,Data e Hora,valor_unitario,quantidade,produto,categoria_produto,nome_evento,PDV,Hora,valor_total,Dia Hora
0,2024-03-09 17:44:43.169,5.0,1,DEPOSITO CAUÇÃO,DEPOSITO CAUÇÃO,TRAPIST - 09/03/2024,PRODUÇÃO CLIENTE,2024-03-09 17,5.0,2024-03-09 17:00:00
1,2024-03-09 17:43:20.032,5.0,1,DEPOSITO CAUÇÃO,DEPOSITO CAUÇÃO,TRAPIST - 09/03/2024,PRODUÇÃO CLIENTE,2024-03-09 17,5.0,2024-03-09 17:00:00
2,2024-03-09 17:49:40.334,5.0,1,DEPOSITO CAUÇÃO,DEPOSITO CAUÇÃO,TRAPIST - 09/03/2024,PRODUÇÃO CLIENTE,2024-03-09 17,5.0,2024-03-09 17:00:00
3,2024-03-09 19:06:33.846,5.0,1,DEPOSITO CAUÇÃO,DEPOSITO CAUÇÃO,TRAPIST - 09/03/2024,CAIXA 2,2024-03-09 19,5.0,2024-03-09 19:00:00
4,2024-03-09 19:06:53.727,5.0,1,DEPOSITO CAUÇÃO,DEPOSITO CAUÇÃO,TRAPIST - 09/03/2024,CAIXA 14,2024-03-09 19,5.0,2024-03-09 19:00:00
...,...,...,...,...,...,...,...,...,...,...
16924,2024-03-10 06:34:18.775,36.0,1,FININHO BACON,NETÃO,TRAPIST - 09/03/2024,PISTA BURGUER+FRITAS 1,2024-03-10 06,36.0,2024-03-10 06:00:00
16925,2024-03-10 06:34:18.775,46.0,1,COMBO SMASH + FRITAS,NETÃO,TRAPIST - 09/03/2024,PISTA BURGUER+FRITAS 1,2024-03-10 06,46.0,2024-03-10 06:00:00
16926,2024-03-10 06:18:23.452,10.0,1,Refrigerante,NÃO ALCOOLICOS,TRAPIST - 09/03/2024,PREMIUM BAR 4 19,2024-03-10 06,10.0,2024-03-10 06:00:00
16927,2024-03-10 06:34:01.570,8.0,2,Água,NÃO ALCOOLICOS,TRAPIST - 09/03/2024,BACKSTAGE BAR 6 29,2024-03-10 06,16.0,2024-03-10 06:00:00


- - - 

# Eventos que rodam em "N" (Tardezinhas e Soweto)

In [32]:
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
from datetime import datetime, timedelta

In [33]:
pd.set_option('display.max_columns', None)

{'codigo_retorno': 0,
 'mensagem_erro': '',
 'dados': {'token_acesso': 'string_de_acesso',
  'data_expiracao': 'expira_em'}}

In [34]:
ID_EVENTOS=[
    15740, #Tardezinha Campinas 2023
    16092, #TADEZINHA CAMPINAS 2023 LOJA
    17990, #Tardezinha RB 2023
    19364, #Trapist
    19404, #Trapist 2024 
    21031, #Soweto Jul24
    21032, # ME LEVA
    21033 #NUMANICE TOUR 3  CAMPINAS


]

DATA_INICIO=[
    "2023-07-15", #Tardezinha Campinas 2023
    "2023-07-15", #TADEZINHA CAMPINAS 2023 LOJA
    "2023-12-02", #Tardezinha RB 2023
    "2024-03-09", #Trapist
    "2024-03-09", #Trapist 2024 
    "2024-07-27", #Soweto Jul24
    "2024-10-19", # ME LEVA
    "2024-10-12" #NUMANICE TOUR 3  CAMPINAS


]

#Definir a Dataframe de produtos (df_produtos)
df_produtos=pd.DataFrame()

for i in range(len(ID_EVENTOS)):
    ################################################ Gerando Token ################################################

    api_url= "https://app.netpdv.com:5566/api/relatorios/GeraToken"

    #tardezinha Campinas 15740
    #tardezinha RB  17990

    
    headers = {
        
        "Content-Type": "text/plain",
        "User-Agent": "PostmanRuntime/7.24.0",
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "token_parceiro": "BCED47DAE6",
        "Content-type": "application/json"
    }


    body = {
        'codigo_evento': ID_EVENTOS[i]
    }




    response_token = requests.post(api_url, headers=headers, json=body)
    token_json=response_token.json()
    token=token_json['dados']['token_acesso']


    ################################################ Fazendo as requisições ################################################


    api_url="https://app.netpdv.com:5566/api/relatorios/ListaTransacoes"


    #deixar setado para inicio às 12hrs e fim às 06hrs do dia seguinte
    #data início sempre deve ser igual à data fim. O código pega as 12hrs da data início e vai até as 06hrs do dia seguinte
    data_inicio=DATA_INICIO[i]

    data_fim=DATA_INICIO[i]

    horario_inicio = "12:00:00"
    horario_fim="13:00:00"




    

    #Loop para pegar as transações (Preciso adicionar um for loop antes para cada evento)
    #ATENÇÃO: Talvez seja necessário gerar outro token

    print(ID_EVENTOS[i])
    while horario_fim!="07:00:00":

        body={
            "data_inicio":data_inicio+" "+horario_inicio,
            "data_fim":data_fim+" "+horario_fim,
        }


        headers = {
            "token_acesso":token
        }

        print(horario_fim)

        ##############################################################
        #Atualizando os horários para o próximo loop
        horario_inicio=pd.to_datetime(horario_inicio, format='%H:%M:%S')
        horario_inicio +=timedelta(hours=1)
        horario_inicio=horario_inicio.strftime('%H:%M:%S')

        horario_fim=pd.to_datetime(horario_fim, format='%H:%M:%S')
        horario_fim +=timedelta(hours=1)
        horario_fim=horario_fim.strftime('%H:%M:%S')



        if horario_fim=="00:00:00":
            data_fim=pd.to_datetime(data_fim, format='%Y-%m-%d')
            data_fim += timedelta(days=1)
            data_fim=data_fim.strftime('%Y-%m-%d')

        if horario_inicio=="00:00:00":
            data_inicio=pd.to_datetime(data_inicio, format='%Y-%m-%d')
            data_inicio += timedelta(days=1)
            data_inicio=data_inicio.strftime('%Y-%m-%d')

        #############################################



        time.sleep(5)

        response_transacoes = requests.post(api_url, headers=headers, json=body)
        transacoes_json=response_transacoes.json()
        produtos = transacoes_json["dados"]["transacoes"]
        df_temp=pd.DataFrame()


        for produto in produtos:
            produto_df=pd.DataFrame(produto)
            infos_adicionais=produto_df[["codigo_evento","nome_evento","data_hora_realizacao","documento_cliente"]]
            caracteristicas=produto["produtos"]
            temporario=pd.DataFrame(caracteristicas)
            temporario=pd.concat([temporario,infos_adicionais], axis=1)
            df_temp=pd.concat([df_temp,temporario], ignore_index=True)

        df_produtos=pd.concat([df_produtos,df_temp], ignore_index=True)

        df_produtos["data_hora_realizacao"]=pd.to_datetime(df_produtos["data_hora_realizacao"])






df_produtos["Dia Hora"]=df_produtos["data_hora_realizacao"].dt.strftime('%Y-%m-%d %H')


15740
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
06:00:00
16092
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
06:00:00
17990
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
06:00:00
19364
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
06:00:00
19404
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
06:00:00
21031
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
0

In [35]:
# Replace the value of 'documento_cliente' if it has less than 3 characters
df_produtos['documento_cliente'] = df_produtos['documento_cliente'].apply(lambda x: np.nan if isinstance(x, str) and len(x) < 3 else x)


In [36]:
df_produtos["nome_evento"].unique()

array(['TARDEZINHA CAMPINAS 2023', 'TADEZINHA CAMPINAS 2023 LOJA',
       'TARDEZINHA RB 2023', 'SOWETO JUL24', 'NUMANICE TOUR 3  CAMPINAS'],
      dtype=object)

In [37]:
df_produtos["Dia Hora"]=df_produtos["data_hora_realizacao"].dt.strftime('%Y-%m-%d %H:%M:%S')

In [38]:
df_produtos["data_hora_realizacao"] = pd.to_datetime(df_produtos["data_hora_realizacao"])

# Aplicando o replace para zerar minutos e segundos
df_produtos["Dia Hora"] = df_produtos["data_hora_realizacao"].apply(lambda x: x.replace(minute=0, second=0))

In [39]:
df_produtos

,categoria_produto,cod_produto,produto,fabricante,quantidade,valor_unitario,valor_total,codigo_evento,nome_evento,data_hora_realizacao,documento_cliente,Dia Hora
0,Bebida,21018,CHOPP,None,1,18.0,18.0,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 12:00:03,ANTONIO,2023-07-15 12:00:00
1,Caixa,None,Ativ. + Rec.,None,1,200.0,200.0,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 12:52:40,28478543805,2023-07-15 12:00:00
2,Caixa,None,Ativ. + Rec.,None,1,43.0,43.0,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 12:59:50,10199815631,2023-07-15 12:00:00
3,Bebida,21018,CHOPP,None,100,18.0,1800.0,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 13:01:19,AVERA,2023-07-15 13:00:00
4,Comida,7374,BACON CHEDDAR,None,1,48.0,48.0,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 13:01:35,28478543805,2023-07-15 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
275882,Bebida,5184,Devolução Produto,None,1,0.0,0.0,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:49:14,43200157895,2024-10-13 00:00:00
275883,Bebida,5184,Devolução Produto,None,1,0.0,0.0,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:49:35,37429670881,2024-10-13 00:00:00
275884,Bebida,5184,Devolução Produto,None,1,0.0,0.0,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:49:41,47143745839,2024-10-13 00:00:00
275885,Bebida,5184,Devolução Produto,None,1,0.0,0.0,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:49:57,47212865893,2024-10-13 00:00:00


In [49]:
df_produtos_concatenado=pd.concat([df_produtos,todos_produtos], ignore_index=True)

In [50]:
df_produtos_concatenado["Unnamed: 0"]=df_produtos_concatenado.index

In [52]:
df_produtos_concatenado.to_excel("produtos_zig.xlsx", index=False, sheet_name="Produtos ZIG")

In [51]:
df_produtos_concatenado

,categoria_produto,cod_produto,produto,fabricante,quantidade,valor_unitario,valor_total,codigo_evento,nome_evento,data_hora_realizacao,documento_cliente,Dia Hora,Data e Hora,PDV,Hora,Unnamed: 0
0,Bebida,21018,CHOPP,None,1,18.0,18.0,15740.0,TARDEZINHA CAMPINAS 2023,2023-07-15 12:00:03,ANTONIO,2023-07-15 12:00:00,NaT,NaN,NaN,0
1,Caixa,None,Ativ. + Rec.,None,1,200.0,200.0,15740.0,TARDEZINHA CAMPINAS 2023,2023-07-15 12:52:40,28478543805,2023-07-15 12:00:00,NaT,NaN,NaN,1
2,Caixa,None,Ativ. + Rec.,None,1,43.0,43.0,15740.0,TARDEZINHA CAMPINAS 2023,2023-07-15 12:59:50,10199815631,2023-07-15 12:00:00,NaT,NaN,NaN,2
3,Bebida,21018,CHOPP,None,100,18.0,1800.0,15740.0,TARDEZINHA CAMPINAS 2023,2023-07-15 13:01:19,AVERA,2023-07-15 13:00:00,NaT,NaN,NaN,3
4,Comida,7374,BACON CHEDDAR,None,1,48.0,48.0,15740.0,TARDEZINHA CAMPINAS 2023,2023-07-15 13:01:35,28478543805,2023-07-15 13:00:00,NaT,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292811,NETÃO,NaN,FININHO BACON,NaN,1,36.0,36.0,NaN,TRAPIST - 09/03/2024,NaT,NaN,2024-03-10 06:00:00,2024-03-10 06:34:18.775,PISTA BURGUER+FRITAS 1,2024-03-10 06,292811
292812,NETÃO,NaN,COMBO SMASH + FRITAS,NaN,1,46.0,46.0,NaN,TRAPIST - 09/03/2024,NaT,NaN,2024-03-10 06:00:00,2024-03-10 06:34:18.775,PISTA BURGUER+FRITAS 1,2024-03-10 06,292812
292813,NÃO ALCOOLICOS,NaN,Refrigerante,NaN,1,10.0,10.0,NaN,TRAPIST - 09/03/2024,NaT,NaN,2024-03-10 06:00:00,2024-03-10 06:18:23.452,PREMIUM BAR 4 19,2024-03-10 06,292813
292814,NÃO ALCOOLICOS,NaN,Água,NaN,2,8.0,16.0,NaN,TRAPIST - 09/03/2024,NaT,NaN,2024-03-10 06:00:00,2024-03-10 06:34:01.570,BACKSTAGE BAR 6 29,2024-03-10 06,292814


In [44]:
todos_produtos

,Data e Hora,valor_unitario,quantidade,produto,categoria_produto,nome_evento,PDV,Hora,valor_total,Dia Hora
0,2024-03-09 17:44:43.169,5.0,1,DEPOSITO CAUÇÃO,DEPOSITO CAUÇÃO,TRAPIST - 09/03/2024,PRODUÇÃO CLIENTE,2024-03-09 17,5.0,2024-03-09 17:00:00
1,2024-03-09 17:43:20.032,5.0,1,DEPOSITO CAUÇÃO,DEPOSITO CAUÇÃO,TRAPIST - 09/03/2024,PRODUÇÃO CLIENTE,2024-03-09 17,5.0,2024-03-09 17:00:00
2,2024-03-09 17:49:40.334,5.0,1,DEPOSITO CAUÇÃO,DEPOSITO CAUÇÃO,TRAPIST - 09/03/2024,PRODUÇÃO CLIENTE,2024-03-09 17,5.0,2024-03-09 17:00:00
3,2024-03-09 19:06:33.846,5.0,1,DEPOSITO CAUÇÃO,DEPOSITO CAUÇÃO,TRAPIST - 09/03/2024,CAIXA 2,2024-03-09 19,5.0,2024-03-09 19:00:00
4,2024-03-09 19:06:53.727,5.0,1,DEPOSITO CAUÇÃO,DEPOSITO CAUÇÃO,TRAPIST - 09/03/2024,CAIXA 14,2024-03-09 19,5.0,2024-03-09 19:00:00
...,...,...,...,...,...,...,...,...,...,...
16924,2024-03-10 06:34:18.775,36.0,1,FININHO BACON,NETÃO,TRAPIST - 09/03/2024,PISTA BURGUER+FRITAS 1,2024-03-10 06,36.0,2024-03-10 06:00:00
16925,2024-03-10 06:34:18.775,46.0,1,COMBO SMASH + FRITAS,NETÃO,TRAPIST - 09/03/2024,PISTA BURGUER+FRITAS 1,2024-03-10 06,46.0,2024-03-10 06:00:00
16926,2024-03-10 06:18:23.452,10.0,1,Refrigerante,NÃO ALCOOLICOS,TRAPIST - 09/03/2024,PREMIUM BAR 4 19,2024-03-10 06,10.0,2024-03-10 06:00:00
16927,2024-03-10 06:34:01.570,8.0,2,Água,NÃO ALCOOLICOS,TRAPIST - 09/03/2024,BACKSTAGE BAR 6 29,2024-03-10 06,16.0,2024-03-10 06:00:00
